In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# MISSING SRULL embeddings from test
import pandas as pd
import re
from tqdm import tqdm
import time
import nltk
import ast
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize
# import tensorflow as tf
# from transformers import XLMRobertaTokenizer, TFXLMRobertaForSequenceClassification, AutoTokenizer
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import pandas as pd
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, mean_squared_error, log_loss, make_scorer
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss,
    confusion_matrix,
    classification_report
)
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/IndiaAI_Hackathon/final_train.csv",low_memory=False)
print(train.shape)

test = pd.read_csv("/content/drive/MyDrive/IndiaAI_Hackathon/final_test.csv",low_memory=False)
print(test.shape)

def create_final_category(val):
  if val in ['Sexually Obscene material', 'Sexually Explicit Act', 'Child Pornography CPChild Sexual Abuse Material CSAM',
 'RapeGang Rape RGRSexually Abusive Content']:
    return 0
  elif val in ['Online Financial Fraud', 'Cryptocurrency Crime', 'Online Gambling  Betting']:
    return 1
  elif val in ['Online and Social Media Related Crime',
 'Any Other Cyber Crime',
 'Cyber Attack/ Dependent Crimes',
 'Hacking  Damage to computercomputer system etc',
 'Cyber Terrorism',
 'Ransomware', 'Report Unlawful Content', 'Online Cyber Trafficking']:
    return 2
  else:
    return 3

train['final_category'] = train['category'].apply(create_final_category)
test['final_category'] = test['category'].apply(create_final_category)

print(train['sub_category'].isna().sum())
print(train['sub_category'].isnull().sum())

print(test['sub_category'].isna().sum())
print(test['sub_category'].isnull().sum())

train['final_subcategory'] = train['sub_category'].copy()
train.loc[train['sub_category'].isnull(), 'final_subcategory'] = train[train['sub_category'].isnull()]['category']
print(train['final_subcategory'].isnull().sum(), train.shape)

test['final_subcategory'] = test['sub_category'].copy()
test.loc[test['sub_category'].isnull(), 'final_subcategory'] = test[test['sub_category'].isnull()]['category']
print(test['final_subcategory'].isnull().sum(), test.shape)

train = train[~train['SRLL_msg_updated'].isnull()].reset_index(drop=True)
print(train.shape)
test = test[~test['SRLL_msg_updated'].isnull()].reset_index(drop=True)
print(test.shape)

category_mappings = {'Cyber Bullying  Stalking  Sexting': 'Cyber Bullying/Stalking/Sexting',
 'Fraud CallVishing': 'Fraud Call/Vishing',
 'Online Gambling  Betting': 'Online Gambling/Betting Fraud',
 'Online Job Fraud': 'Online Job Fraud',
 'UPI Related Frauds': 'UPI-Related Frauds',
 'Internet Banking Related Fraud': 'Internet Banking-Related Fraud',
 'RapeGang Rape RGRSexually Abusive Content': 'Rape/Gang Rape-Sexually Abusive Content',
 'Other': 'Any Other Cyber Crime',
 'Profile Hacking Identity Theft': 'Profile Hacking/Identity Theft',
 'DebitCredit Card FraudSim Swap Fraud': 'Debit/Credit Card/Sim Swap Fraud',
 'EWallet Related Fraud': 'E-Wallet Related Frauds',
 'Data Breach/Theft': 'Data Breach/Theft',
 'Denial of Service (DoS)/Distributed Denial of Service (DDOS) attacks': 'Denial of Service (DoS) and Distributed Denial of Service (DDoS) attacks',
 'FakeImpersonating Profile': 'Impersonation Fraud',
 'Cryptocurrency Fraud': 'Cryptocurrency Fraud',
 'Sexually Explicit Act': 'Sale,Publishing and Transmitting Obscene Material/Sexually Explicit Material',
 'Sexually Obscene material': 'Sale,Publishing and Transmitting Obscene Material/Sexually Explicit Material',
 'Malware Attack': 'Malicious code attacks',
 'Business Email CompromiseEmail Takeover': 'Business Email Compromise/Email Takeover',
 'Email Hacking': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'Cheating by Impersonation': 'Impersonation Fraud',
 'Hacking/Defacement': 'Hacking/Website Defacement',
 'Unauthorised AccessData Breach': 'Unauthorized Access/Data Breach',
 'SQL Injection': 'Malicious code attacks',
 'Provocative Speech for unlawful acts': 'Provocative Speech of Unlawful Acts',
 'Ransomware Attack': 'Malicious code attacks',
 'Cyber Terrorism': 'Cyber Terrorism',
 'Child Pornography CPChild Sexual Abuse Material CSAM': 'Child Pornography/Child Sexual Abuse Material(CSAM)',
 'Tampering with computer source documents': 'Tampering with Computer Source Documents',
 'DematDepository Fraud': 'Demat/Depository Fraud',
 'Online Trafficking': 'Online Cyber Trafficking',
 'Online Matrimonial Fraud': 'Online Matrimonial Fraud',
 'Website DefacementHacking': 'Hacking/Website Defacement',
 'Damage to computer computer systems etc': 'Damage to Computer Systems',
 'Impersonating Email': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'EMail Phishing': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'Ransomware': 'Malicious code attacks',
 'Intimidating Email': 'Email Hacking/Phishing/Impersonating/Intimidating',
 'Against Interest of sovereignty or integrity of India': 'Provocative Speech of Unlawful Acts'}

print(test.shape)
test = test[test['final_subcategory'].isin(train['final_subcategory'])].reset_index(drop=True)
print(test.shape)

train['final_subcategory_grouped'] = train['final_subcategory'].replace(category_mappings)
test['final_subcategory_grouped'] = test['final_subcategory'].replace(category_mappings)

(84040, 13)
(28328, 13)
3798
3798
1358
1358
0 (84040, 14)
0 (28328, 14)
(84023, 14)
(28327, 14)
(28327, 14)
(28323, 14)


## Helper

In [ ]:
def post_analysis(df,x_train, x_test, oot, model, overall_feature_bow, x_train_bow, x_test_bow, oot_bow, model_name = 'lr', prediction_feature = 'final_category'):
  from sklearn.metrics import (roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, log_loss, confusion_matrix,classification_report)

  y_test_pred = model.predict(x_test_bow)
  y_test_proba = model.predict_proba(x_test_bow)[:, 1]
  y_train_pred = model.predict(x_train_bow)
  y_train_proba = model.predict_proba(x_train_bow)[:, 1]
  y_overall_pred = model.predict(overall_feature_bow)
  y_overall_proba = model.predict_proba(overall_feature_bow)[:, 1]
  y_oot_pred = model.predict(oot_bow)
  y_oot_proba = model.predict_proba(oot_bow)[:, 1]

  df[model_name+'_model_score'] = model.predict_proba(overall_feature_bow)[:, 1]
  x_train[model_name+'_model_score'] = model.predict_proba(x_train_bow)[:, 1]
  x_test[model_name+'_model_score'] = model.predict_proba(x_test_bow)[:, 1]
  oot[model_name+'_model_score'] = model.predict_proba(oot_bow)[:, 1]

  # Calculate relevant metrics for the training set
  train_metrics = {
      # "AUC": roc_auc_score(y_train, y_train_proba),
      "Accuracy": accuracy_score(y_train, y_train_pred),
      # "Precision": precision_score(y_train, y_test_pred),
      # "Recall": recall_score(y_train, y_test_pred),
      "F1 Score": f1_score(y_train, y_train_pred, average='weighted'),
      # "Log Loss": log_loss(y_train, y_train_proba)
  }

  test_metrics = {
      # "AUC": roc_auc_score(y_test, y_test_proba),
      "Accuracy": accuracy_score(y_test, y_test_pred),
      # "Precision": precision_score(y_test, y_test_pred),
      # "Recall": recall_score(y_test, y_test_pred),
      "F1 Score": f1_score(y_test, y_test_pred, average='weighted'),
      # "Log Loss": log_loss(y_test, y_test_proba)
  }

  overall_metrics = {
      # "AUC": roc_auc_score(df[prediction_feature].tolist(), y_overall_proba),
      "Accuracy": accuracy_score(df[prediction_feature].tolist(), y_overall_pred),
      # "Precision": precision_score(df[prediction_feature].tolist(), y_overall_pred),
      # "Recall": recall_score(df[prediction_feature].tolist(), y_overall_pred),
      "F1 Score": f1_score(df[prediction_feature].tolist(), y_overall_pred, average='weighted'),
      # "Log Loss": log_loss(df[prediction_feature].tolist(), y_overall_proba)
  }

  oot_metrics = {
      # "AUC": roc_auc_score(oot[prediction_feature].tolist(), y_oot_proba),
      "Accuracy": accuracy_score(oot[prediction_feature].tolist(), y_oot_pred),
      # "Precision": precision_score(oot[prediction_feature].tolist(), y_oot_pred),
      # "Recall": recall_score(oot[prediction_feature].tolist(), y_oot_pred),
      "F1 Score": f1_score(oot[prediction_feature].tolist(), y_oot_pred, average='weighted'),
      # "Log Loss": log_loss(oot[prediction_feature].tolist(), y_oot_proba)
  }

  # Create a DataFrame to compare metrics
  metrics_df = pd.DataFrame({
      'Metric': list(train_metrics.keys()),
      'Train': list(train_metrics.values()),
      'Test': list(test_metrics.values()),
      'Overall Data': list(overall_metrics.values()),
      'OOT': list(oot_metrics.values())
  })

  # Display the DataFrame
  return display(metrics_df)

In [ ]:
def classwise_scores_and_overall(model, X_test_embed, y_test):

  y_prob = model.predict_proba(X_test_embed)
  y_pred = model.predict(X_test_embed)
  y_test_bin = label_binarize(y_test, classes=[0, 1, 2])

  # Calculate per-class metrics
  metrics = []
  for i, class_label in enumerate(nb.classes_):
      class_auc = roc_auc_score(y_test_bin[:, i], y_prob[:, i])
      class_accuracy = accuracy_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_f1 = f1_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_recall = recall_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_precision = precision_score((y_test == class_label).astype(int), (y_pred == class_label).astype(int))
      class_loss = log_loss(y_test_bin[:, i], y_prob[:, i])

      metrics.append({
          'Class': class_label,
          'AUC': class_auc,
          'Accuracy': class_accuracy,
          'F1 Score': class_f1,
          'Recall': class_recall,
          'Precision': class_precision,
          'Log Loss': class_loss
      })

  metrics_df = pd.DataFrame(metrics)

  # Calculate overall metrics
  overall_accuracy = accuracy_score(y_test, y_pred)
  overall_f1 = f1_score(y_test, y_pred, average='weighted')
  overall_recall = recall_score(y_test, y_pred, average='weighted')
  overall_precision = precision_score(y_test, y_pred, average='weighted')
  overall_auc = roc_auc_score(y_test_bin, y_prob, average='weighted', multi_class='ovr')
  overall_loss = log_loss(y_test_bin, y_prob)

  # Display overall metrics
  overall_metrics = {
      'Overall Accuracy': overall_accuracy,
      'Overall F1 Score': overall_f1,
      'Overall Recall': overall_recall,
      'Overall Precision': overall_precision,
      'Overall AUC': overall_auc,
      'Overall Log Loss': overall_loss
  }
  overall_metrics_df = pd.DataFrame([overall_metrics])
  return metrics_df, overall_metrics_df

# Category Classification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    train['SRLL_msg_updated'],
    train['final_category'], test_size=0.25, random_state=42, stratify=train['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(63017,) (21006,)
(63017,) (21006,)


## Naive Bayes

### BOW with max features=1000

In [ ]:
# BOW
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
y_oot = test1['final_category'].copy()
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])
y_train_overall = train1['final_category'].copy()

nb = MultinomialNB(alpha: 0.1, fit_prior= True)
nb.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(X_train_overall_bow, y_train_overall)

classwise = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])
classwise.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall.index = ['train', 'test', 'train_overall','oot']

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)


In [ ]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.716544,0.715653,0.716321,0.715305
Overall F1 Score,0.572721,0.573925,0.573018,0.579953
Overall Recall,0.692866,0.695545,0.693536,0.698612
Overall Precision,0.591011,0.591189,0.591051,0.596010
Overall AUC,0.860775,0.859769,0.860524,0.861318
Overall Log Loss,1.959407,1.958223,1.959111,1.958106


### TF-IDF with max features=1000

In [ ]:
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

nb = MultinomialNB(alpha= 0.1, fit_prior= True)
nb.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(nb,X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(nb,X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(nb,X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(nb,X_oot_tfidf, y_oot)

classwise = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
classwise.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall.index = ['train', 'test', 'train_overall','oot']

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)


In [ ]:
overall.T

,train,test,train_overall,oot
Overall Accuracy,0.776847,0.773303,0.775961,0.771029
Overall F1 Score,0.768662,0.764676,0.767667,0.761781
Overall Recall,0.776847,0.773303,0.775961,0.771029
Overall Precision,0.766759,0.762655,0.765735,0.759696
Overall AUC,0.889154,0.884533,0.887999,0.887112
Overall Log Loss,0.487744,0.496697,0.489982,0.496444


## Random Forest

### BOW with max features=1000

In [ ]:
# BOW
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

y_train_overall = train1['final_category'].copy()
y_oot = test1['final_category'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise_rf_bow = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])
classwise_rf_bow.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_rf_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_rf_bow.index = ['train', 'test', 'train_overall','oot']

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)


In [ ]:
overall_rf_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.996541,0.800010,0.947406,0.800939
Overall F1 Score,0.995406,0.572631,0.916487,0.602867
Overall Recall,0.993039,0.558296,0.884332,0.576671
Overall Precision,0.997808,0.774764,0.959037,0.804031
Overall AUC,0.999642,0.901815,0.987458,0.905616
Overall Log Loss,0.122826,0.509708,0.219550,0.490786


### TF-IDF with max features=1000

In [ ]:
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_rf_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
classwise_rf_tfidf.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_rf_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_rf_tfidf.index = ['train', 'test', 'train_overall','oot']

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)


In [ ]:
overall_rf_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.996287,0.799391,0.947061,0.799597
Overall F1 Score,0.995212,0.567642,0.916231,0.596362
Overall Recall,0.992902,0.554835,0.883364,0.572194
Overall Precision,0.997555,0.801843,0.959975,0.810421
Overall AUC,0.999623,0.906116,0.988275,0.908346
Overall Log Loss,0.122744,0.484708,0.213238,0.488975


## XGBoost

## LightGBM

In [ ]:
from lightgbm import LGBMClassifier

# BOW
feature_name = 'SLL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

y_train_overall = train1['final_category'].copy()
y_oot = test1['final_category'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise_lgbm_bow = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])
classwise_lgbm_bow.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_lgbm_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_lgbm_bow.index = ['train', 'test', 'train_overall','oot']

In [ ]:
overall_lgbm_bow.T

In [ ]:
feature_name = 'SLL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_lgbm_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
classwise_lgbm_tfidf.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_lgbm_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_lgbm_tfidf.index = ['train', 'test', 'train_overall','oot']

In [ ]:
overall_lgbm_tfidf.T

## LR

In [ ]:
# TFIDF - SRLL
feature_name = 'SRLL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

model = LogisticRegression(C=0.1, random_state=42, class_weight=None, penalty='l2', solver='lbfgs')
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_lr_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
classwise_lr_tfidf.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_lr_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_lr_tfidf.index = ['train', 'test', 'train_overall','oot']

(84023, 15) (84023, 15)
(28323, 15) (28323, 15)
(63017,) (21006,)
(63017,) (21006,)


In [ ]:
overall_lr_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.801419,0.792440,0.799174,0.790665
Overall F1 Score,0.786173,0.777867,0.784096,0.773671
Overall Recall,0.801419,0.792440,0.799174,0.790665
Overall Precision,0.796535,0.785392,0.793662,0.783860
Overall AUC,0.906158,0.899059,0.904389,0.901725
Overall Log Loss,0.455644,0.468435,0.458842,0.467517


In [ ]:
# TFIDF - SRLL
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

model = LogisticRegression(C=0.1, random_state=42, class_weight=None, penalty='l2', solver='lbfgs')
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_lr_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
classwise_lr_tfidf.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_lr_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_lr_tfidf.index = ['train', 'test', 'train_overall','oot']

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)


In [ ]:
overall_lr_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.800714,0.794725,0.799217,0.791193
Overall F1 Score,0.785600,0.779835,0.784160,0.774215
Overall Recall,0.800714,0.794725,0.799217,0.791193
Overall Precision,0.794817,0.787991,0.793097,0.784564
Overall AUC,0.905496,0.900869,0.904340,0.901815
Overall Log Loss,0.456878,0.464848,0.458871,0.467330


## CatBoost

In [ ]:
from catboost import CatBoostClassifier

# BOW with SRLL
feature_name = 'SRLL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

y_train_overall = train1['final_category'].copy()
y_oot = test1['final_category'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise_cat_bow = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])
classwise_cat_bow.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_cat_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_cat_bow.index = ['train', 'test', 'train_overall','oot']

(84023, 15) (84023, 15)
(28323, 15) (28323, 15)
(63017,) (21006,)
(63017,) (21006,)


In [ ]:
overall_cat_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.789676,0.780110,0.787284,0.783144
Overall F1 Score,0.550013,0.540028,0.547514,0.537608
Overall Recall,0.539771,0.531219,0.537633,0.532628
Overall Precision,0.764481,0.723751,0.753863,0.710935
Overall AUC,0.897269,0.891580,0.895854,0.894608
Overall Log Loss,0.485347,0.495095,0.487784,0.496090


In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_cat_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
classwise_cat_tfidf.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_cat_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_cat_tfidf.index = ['train', 'test', 'train_overall','oot']

(84023, 15) (84023, 15)
(28323, 15) (28323, 15)
(63017,) (21006,)
(63017,) (21006,)


In [ ]:
overall_cat_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.791501,0.784633,0.789784,0.785439
Overall F1 Score,0.554566,0.548875,0.553140,0.541694
Overall Recall,0.543893,0.538981,0.542665,0.536270
Overall Precision,0.767236,0.733542,0.758233,0.717836
Overall AUC,0.897576,0.893601,0.896590,0.895411
Overall Log Loss,0.485376,0.493250,0.487344,0.495920


In [ ]:
from catboost import CatBoostClassifier

# BOW with SRULL
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

y_train_overall = train1['final_category'].copy()
y_oot = test1['final_category'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise_cat_bow = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])
classwise_cat_bow.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_cat_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_cat_bow.index = ['train', 'test', 'train_overall','oot']

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)


In [ ]:
overall_cat_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.788352,0.783014,0.787018,0.783212
Overall F1 Score,0.548875,0.544202,0.547710,0.536276
Overall Recall,0.538563,0.533821,0.537377,0.531755
Overall Precision,0.764711,0.735267,0.756798,0.713656
Overall AUC,0.897146,0.894767,0.896546,0.894837
Overall Log Loss,0.486475,0.490731,0.487539,0.496562


In [ ]:
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_cat_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
classwise_cat_tfidf.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_cat_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_cat_tfidf.index = ['train', 'test', 'train_overall','oot']

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)


In [ ]:
overall_cat_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.791494,0.785775,0.790064,0.785119
Overall F1 Score,0.555831,0.548083,0.553895,0.537006
Overall Recall,0.544677,0.538203,0.543058,0.534134
Overall Precision,0.772512,0.743624,0.765212,0.696415
Overall AUC,0.897258,0.894639,0.896594,0.895202
Overall Log Loss,0.485789,0.489805,0.486793,0.495224


## Gradient Boosting

In [ ]:
# BOW
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

y_train_overall = train1['final_category'].copy()
y_oot = test1['final_category'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

model = GradientBoostingClassifier()
model.fit(X_train_bow, y_train)

train_class_wise_bow, train_overall_bow = classwise_scores_and_overall(model, X_train_bow, y_train)
test_class_wise_bow, test_overall_bow = classwise_scores_and_overall(model, X_test_bow, y_test)
oot_class_wise_bow, oot_overall_bow = classwise_scores_and_overall(model,X_oot_bow, y_oot)
train_overall_class_wise_bow, train_overall_overall_bow = classwise_scores_and_overall(model,X_train_overall_bow, y_train_overall)

classwise_gb_bow = pd.concat([train_class_wise_bow, test_class_wise_bow, train_overall_class_wise_bow, oot_class_wise_bow])
classwise_gb_bow.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_gb_bow = pd.concat([train_overall_bow, test_overall_bow, train_overall_overall_bow, oot_overall_bow])
overall_gb_bow.index = ['train', 'test', 'train_overall','oot']

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)


In [ ]:
overall_gb_bow.T

,train,test,train_overall,oot
Overall Accuracy,0.793541,0.782824,0.790862,0.785154
Overall F1 Score,0.612880,0.590293,0.607253,0.591028
Overall Recall,0.579419,0.561888,0.575036,0.564082
Overall Precision,0.751889,0.714431,0.742689,0.727501
Overall AUC,0.900365,0.894496,0.898885,0.894253
Overall Log Loss,0.479650,0.490235,0.482296,0.495845


In [ ]:
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

model = GradientBoostingClassifier()
model.fit(X_train_tfidf, y_train)

train_class_wise_tfidf, train_overall_tfidf = classwise_scores_and_overall(model, X_train_tfidf, y_train)
test_class_wise_tfidf, test_overall_tfidf = classwise_scores_and_overall(model, X_test_tfidf, y_test)
train_overall_class_wise_tfidf, train_overall_overall_tfidf = classwise_scores_and_overall(model, X_train_overall_tfidf, y_train_overall)
oot_class_wise_tfidf, oot_overall_tfidf = classwise_scores_and_overall(model, X_oot_tfidf, y_oot)

classwise_gb_tfidf = pd.concat([train_class_wise_tfidf, test_class_wise_tfidf, train_overall_class_wise_tfidf, oot_class_wise_tfidf])
classwise_gb_tfidf.index = ['train_0', 'train_1', 'train_2', 'test_0', 'test_1', 'test_2', 'train_overall_0','train_overall_1','train_overall_2','oot_0', 'oot_1', 'oot_2']

overall_gb_tfidf = pd.concat([train_overall_tfidf, test_overall_tfidf, train_overall_overall_tfidf, oot_overall_tfidf])
overall_gb_tfidf.index = ['train', 'test', 'train_overall','oot']

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)


In [ ]:
overall_gb_tfidf.T

,train,test,train_overall,oot
Overall Accuracy,0.799238,0.787013,0.796182,0.786320
Overall F1 Score,0.626224,0.595781,0.618671,0.589792
Overall Recall,0.590448,0.567341,0.584670,0.564495
Overall Precision,0.770299,0.721956,0.758707,0.717781
Overall AUC,0.904916,0.895474,0.902535,0.895731
Overall Log Loss,0.470613,0.487199,0.474760,0.491811


## SVC

In [ ]:
feature_name = 'SRLL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

# Training Logistic Regression model with BoW
bow_model = SVC(kernel='rbf', gamma='scale', random_state=42)
bow_model.fit(X_train_bow, y_train)

X_oot = bow_vectorizer.transform(test1[feature_name])
X_train_overall = bow_vectorizer.transform(train1[feature_name])
y_oot_pred =  bow_model.predict(X_oot)
y_oot = test1['final_category'].copy()


post_analysis(train1, X_train, X_test, test1, bow_model, X_train_overall,
                X_train_bow, X_test_bow, X_oot, model_name = 'lr',
                prediction_feature = 'final_category')

(84023, 15) (84023, 15)
(28323, 15) (28323, 15)
(63017,) (21006,)
(63017,) (21006,)


## Hyperparameter tuning of LR SRLL-TFIDF

In [ ]:
s = time.time()
feature_name = 'SRLL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

# tfidf_vectorizer = TfidfVectorizer(max_features=1000)
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)
# X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
# X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
# y_oot = test1['final_category'].copy()
# y_train_overall = train1['final_category'].copy()

lr = LogisticRegression(multi_class='multinomial', max_iter=1000)
param_grid = {
    'C': [0.1, 1, 10],       # Regularization strength
    'penalty': ['l2'],                          # 'l2' penalty works with 'lbfgs' and 'newton-cg' solvers for multinomial
    'solver': ['lbfgs', 'newton-cg', 'saga'],   # Solvers that support 'multinomial' with L2 penalty
    # 'max_iter': [500, 1000, 1500],
    'class_weight': [None, 'balanced'],
    # 'tol': [1e-5, 1e-4]
}

grid_search = GridSearchCV(
    estimator=lr,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit grid search to the data
grid_search.fit(X_train_tfidf, y_train)

# Best parameters found by GridSearch
print(f"Best parameters found: {grid_search.best_params_}")
e = time.time()
print(f"{(e-s)/60} mins")

(84023, 15) (84023, 15)
(28323, 15) (28323, 15)
(63017,) (21006,)
(63017,) (21006,)
Fitting 5 folds for each of 18 candidates, totalling 90 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Best parameters found: {'C': 0.1, 'class_weight': None, 'penalty': 'l2', 'solver': 'lbfgs'}
16.890313975016277 mins


In [ ]:
{'C': 0.1, 'class_weight': None, 'penalty': 'l2', 'solver': 'lbfgs'}

## Hyperparameter tuning for LR-TFIDF-SRULL

In [ ]:
s = time.time()
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

lr = LogisticRegression(multi_class='multinomial', max_iter=1000)
param_grid = {
    'C': [0.1, 1, 10],       # Regularization strength
    'penalty': ['l2'],                          # 'l2' penalty works with 'lbfgs' and 'newton-cg' solvers for multinomial
    'solver': ['lbfgs', 'newton-cg', 'saga'],   # Solvers that support 'multinomial' with L2 penalty
    # 'max_iter': [500, 1000, 1500],
    'class_weight': [None, 'balanced'],
    # 'tol': [1e-5, 1e-4]
}

grid_search = GridSearchCV(
    estimator=lr,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Fit grid search to the data
grid_search.fit(X_train_tfidf, y_train)

# Best parameters found by GridSearch
print(f"Best parameters found: {grid_search.best_params_}")
e = time.time()
print(f"{(e-s)/60} mins")

{'C': 0.1, 'class_weight': None, 'penalty': 'l2', 'solver': 'lbfgs'}

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)
Fitting 5 folds for each of 18 candidates, totalling 90 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Best parameters found: {'C': 0.1, 'class_weight': None, 'penalty': 'l2', 'solver': 'lbfgs'}
19.854378640651703 mins


## Hyperparameter for Naive Bayes SRULL-TFIDF

In [ ]:
s = time.time()
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
    'fit_prior': [True, False]
}

nb_model = MultinomialNB()
grid_search = GridSearchCV(estimator=nb_model, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=2, n_jobs=-1)

# Perform grid search on the training data
grid_search.fit(X_train_tfidf, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

e = time.time()
print(f"{(e-s)/60} mins")

{'alpha': 0.1, 'fit_prior': True}

(84021, 15) (84023, 15)
(28318, 15) (28323, 15)
(63015,) (21006,)
(63015,) (21006,)
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'alpha': 0.1, 'fit_prior': True}
Best F1 Score: nan
0.14840449889500937 mins


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


## Hyperparameter tuning on CatBoost

In [ ]:
s = time.time()
feature_name = 'SRLL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_oot_tfidf = tfidf_vectorizer.transform(test1[feature_name])
X_train_overall_tfidf = tfidf_vectorizer.transform(train1[feature_name])
y_oot = test1['final_category'].copy()
y_train_overall = train1['final_category'].copy()

# Define the CatBoost model
catboost_model = CatBoostClassifier(learning_rate=0.1, depth=6, iterations=100, verbose=0, random_seed=42)

# Define the parameter grid for tuning
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    # 'depth': [4, 6, 8],
    # 'l2_leaf_reg': [1, 3, 5],  # Regularization parameter
    # 'border_count': [32, 64, 128]  # Number of splits for numerical features
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=10, n_jobs=-1)

# Perform grid search on the training data
grid_search.fit(X_train_tfidf, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)
e = time.time()
print(f"{(e-s)/60} mins")

In [ ]:
s = time.time()
feature_name = 'SRULL_msg_updated'
train1 = train[~train[feature_name].isnull()].reset_index(drop=True)
print(train1.shape, train.shape)
test1 = test[~test[feature_name].isnull()].reset_index(drop=True)
print(test1.shape, test.shape)

X_train, X_test, y_train, y_test = train_test_split(
    train1[feature_name],
    train1['final_category'], test_size=0.25, random_state=42, stratify=train1['final_category']
    )
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

y_train_overall = train1['final_category'].copy()
y_oot = test1['final_category'].copy()

bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
X_oot_bow = bow_vectorizer.transform(test1[feature_name])
X_train_overall_bow = bow_vectorizer.transform(train1[feature_name])

# Define the CatBoost model
catboost_model = CatBoostClassifier(random_seed=42)

# Define the parameter grid for tuning
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5],  # Regularization parameter
    'border_count': [32, 64, 128]  # Number of splits for numerical features
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=10, n_jobs=-1)

# Perform grid search on the training data
grid_search.fit(X_train_bow, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)
e = time.time()
print(f"{(e-s)/60} mins")

In [ ]:
# Training Logistic Regression model with BoW
lr_bow = LogisticRegression()
lr_bow.fit(X_train_bow, y_train)

# Making predictions
y_pred_bow = lr_bow.predict(X_test_bow)

X_oot = bow_vectorizer.transform(test['SRLL_msg_updated'])
X_train_overall = bow_vectorizer.transform(train['SRLL_msg_updated'])
y_oot_pred =  lr_bow.predict(X_oot)
y_oot = test['final_category'].copy()


post_analysis(train, X_train, X_test, test, lr_bow, X_train_overall,
                X_train_bow, X_test_bow, X_oot, model_name = 'lr',
                prediction_feature = 'final_category')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Metric,Train,Test,Overall Data,OOT
0,Accuracy,0.802498,0.784538,0.798008,0.786746
1,F1 Score,0.792223,0.773834,0.787615,0.774557


In [ ]:
# Using Bag of Words (BoW)
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Training Logistic Regression model with BoW
lr_tfidf = LogisticRegression()
lr_tfidf.fit(X_train_tfidf, y_train)

# Making predictions
y_pred_tfidf = lr_tfidf.predict(X_test_tfidf)

print("BoW Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_tfidf))

X_oot = tfidf_vectorizer.transform(test['SRLL_msg_updated'])
y_oot_pred =  lr_tfidf.predict(X_oot)
y_oot = test['final_category'].copy()
print(classification_report(y_oot, y_oot_pred))

a = pd.Series(tfidf_vectorizer.vocabulary_).reset_index()
a.columns = ['word','count']
# a.sort_values('count', ascending=False)

BoW Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.21      0.31       949
           1       0.84      0.89      0.87     13023
           2       0.70      0.68      0.69      7034

    accuracy                           0.79     21006
   macro avg       0.70      0.59      0.62     21006
weighted avg       0.78      0.79      0.78     21006

              precision    recall  f1-score   support

           0       0.61      0.20      0.30      1357
           1       0.84      0.90      0.87     17361
           2       0.71      0.68      0.69      9605

    accuracy                           0.79     28323
   macro avg       0.72      0.59      0.62     28323
weighted avg       0.78      0.79      0.78     28323

